In [39]:
import numpy as np
import os
import string
import random
import csv

# Core Functions

In [40]:
'''
This is what we use to convert a variable to a binary sring.
Takes in the key, then uses format to format it to a 16 len binary string.
Important to not that it is explicitly a string, not an int.
'''
def hash_funtion(key):
    
    return  '{0:016b}'.format(key)

#
class Bucket:
    
    def __init__(self,local_depth,index,empty_spaces,id):
        
        self.id = id
        self.local_depth = local_depth
        self.index = index
        self.empty_spaces = empty_spaces

class Directory:
    
    def  __init__(self,global_depth,directory_records):
        
        self.global_depth = global_depth,
        self.directory_records = directory_records

class DirectoryRecord:
    
    def __init__(self,bucket, hash_prefix):
        
        self.hash_prefix = hash_prefix
        self.value = bucket

# Hyper Parameters

In [41]:
bucket_capacity = 2
bucket_number = 3
global_depth = 2

In [42]:
# Initialization of buckets
bucket1 = Bucket(local_depth = 1, empty_spaces = bucket_capacity, index = [], id = 1)
bucket2 = Bucket(local_depth = 1, empty_spaces = bucket_capacity, index = [], id = 2)

# Initialization of directory
directory_records = list()  

directory_records.append(DirectoryRecord(hash_prefix = 0, bucket = bucket1))
directory_records.append(DirectoryRecord(hash_prefix = 1, bucket = bucket2))

directory = Directory(global_depth = 1, directory_records = directory_records)

In [43]:
directory_records

In [44]:
directory.directory_records[1].hash_prefix

1

# Insertion Algorithm

In [45]:
def insert(index):
    
    global directory
    global bucket_number
    
    t_id = index[0]
    hash_key = hash_funtion(int(t_id))
    
    hash_prefix = int(hash_key[-directory.global_depth[0]:], 2)

    bucket = directory.directory_records[hash_prefix].value
    bucket.index.append(index)
    bucket.empty_spaces = int(bucket.empty_spaces)-1

    if(bucket.empty_spaces < 0):

        tempopary_memory = bucket.index    
        bucket.empty_spaces = bucket_capacity
        bucket.index = []

        if (directory.global_depth[0] > bucket.local_depth):

            # NUMBER OF LINKED BUCKETS
            number_of_links = 2*(directory.global_depth[0] - bucket.local_depth)
            print("Woo",number_of_links)
            bucket.local_depth = bucket.local_depth + 1
            number_of_modify_links = number_of_links/2 

            new_bucket = Bucket(local_depth = bucket.local_depth, index=[], empty_spaces = bucket_capacity, id = bucket_number)

            for directory_record in directory.directory_records:

                if(directory_record.value == bucket):
                    if(number_of_modify_links != 0):
                        number_of_modify_links = number_of_modify_links - 1
                    else:
                        directory_record.value = new_bucket
                        bucket_number = bucket_number + 1

            for i in range(len(tempopary_memory)):
                insert(tempopary_memory[i])
                

        elif (directory.global_depth[0] == bucket.local_depth):
            
            new_directory_len = 2 * len(directory.directory_records)
            new_directory_records = []

            for directory_record_number in range(new_directory_len):
                new_directory_records.append(DirectoryRecord(hash_prefix=directory_record_number,bucket=Bucket(local_depth=1,index=[],empty_spaces=bucket_capacity,id=bucket_number)))
                #bucket_number = bucket_number + 1
            
            new_directory = Directory(global_depth=directory.global_depth[0]+1,directory_records=new_directory_records)

            # REHASING

            for directory_record in directory.directory_records:
                haskey1 = '0'+hash_funtion(directory_record.hash_prefix)
                haskey2 = '1'+hash_funtion(directory_record.hash_prefix)
                new_index1 = int(haskey1[-directory.global_depth[0]:],2)
                new_index2 = int(haskey2[-directory.global_depth[0]:],2)

                new_directory.directory_records[new_index1].value = directory_record.value
                new_directory.directory_records[new_index2].value = directory_record.value

            directory= new_directory

            for i in range(len(tempopary_memory)):

                insert(tempopary_memory[i])
    


# Evidence of running the code

In [46]:
t_id = 0
t_amount = 100
u_name = 'David'

insert([t_id, t_amount, u_name])
t_id = 1
t_amount = 101
u_name = 'David2'
insert([t_id, t_amount, u_name])

In [47]:
directory_records

In [48]:
directory.directory_records[0].value.index

[[0, 100, 'David']]

In [49]:
t_id = 2
t_amount = 102
u_name = 'David2'
insert([t_id, t_amount, u_name])

In [50]:
directory_records

In [51]:
directory.directory_records[0].value.index[0]

[0, 100, 'David']

In [52]:
directory.directory_records[0].value.local_depth

1

This to me seems like evidence that something is not functional, the bucket should split since it's local depth is one yet it is soring 2 values...

In [53]:
t_id = 4
t_amount = 102
u_name = 'David2'
insert([t_id, t_amount, u_name])

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[15], line 4
      2 t_amount = 102
      3 u_name = 'David2'
----> 4 insert([t_id, t_amount, u_name])

Cell In[7], line 69
     65 directory= new_directory
     67 for i in range(len(tempopary_memory)):
---> 69     insert(tempopary_memory[i],lock)

NameError: name 'lock' is not defined
---------------------------------------------------------------------------
Hmm something is broken here... what is lock and why isnt it defined...

In [54]:
directory.directory_records

In [55]:
for i in range(len(directory.directory_records)):
    print(hash_funtion(directory.directory_records[i].hash_prefix))
    print(directory.directory_records[i].value.id)
    print(directory.directory_records[i].value.local_depth)
    print(directory.directory_records[i].value.index)


0000000000000000
1
1
[[0, 100, 'David'], [4, 102, 'David2']]
0000000000000001
2
1
[[1, 101, 'David2']]
0000000000000010
3
1
[[2, 102, 'David2']]
0000000000000011
3
1
[]


0000000000000000
1
1
[[0, 100, 'David'], [4, 102, 'David2']]
0000000000000001
2
1
[[1, 101, 'David2']]
0000000000000010
5
1
[[2, 102, 'David2']]
0000000000000011
6
1
[]

In [56]:
t_id = 8
t_amount = 102
u_name = 'David2'
insert([t_id, t_amount, u_name])

Woo 2


In [57]:
for i in range(len(directory.directory_records)):
    print(hash_funtion(directory.directory_records[i].hash_prefix))
    print("ID: ",directory.directory_records[i].value.id)
    print(directory.directory_records[i].value.local_depth)
    print(directory.directory_records[i].value.index)

0000000000000000
ID:  1
2
[[0, 100, 'David'], [8, 102, 'David2']]
0000000000000001
ID:  2
1
[[1, 101, 'David2']]
0000000000000010
ID:  3
1
[[2, 102, 'David2']]
0000000000000011
ID:  3
1
[]
0000000000000100
ID:  3
1
[[4, 102, 'David2']]
0000000000000101
ID:  3
1
[]
0000000000000110
ID:  3
1
[]
0000000000000111
ID:  3
1
[]
